## 5. Regression Models

In [ ]:
# Enable R cell for later use
%load_ext rpy2.ipython

In [ ]:
import cmocean
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from feature_engineering import feature_engineer, drop_cols
from grid_search import get_best_model
from mlxtend.regressor import StackingRegressor, StackingCVRegressor
from model_comparer import ModelComparer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

%matplotlib inline

In [ ]:
# Read data from the data folder
file_directory = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '')) + '\\'
sample = False
sample_size = 10000
        
max_window = 3

try:
    df_combined = pd.read_csv(file_directory + 'data/horse_race_combined.csv', low_memory=False, index_col=0)
    df_combined['run_date'] = df_combined['run_date'].apply(lambda x: pd.Timestamp(x))
    df_combined = df_combined.sort_values(['horse_id', 'run_date'])
    
    if sample:
        df_combined = df_combined.sort_values('run_date').iloc[:sample_size]
    
    df_combined.set_index(['horse_id', 'run_date'], inplace=True)
    
    first_occur_df = pd.read_csv(file_directory + 'data/first_occurence_race.csv', low_memory=False, index_col=0)
    first_occur_df['run_date'] = first_occur_df['run_date'].apply(pd.Timestamp)
    first_occur_df = first_occur_df.sort_values(['horse_id', 'run_date'])
    drop_cols(first_occur_df)
    
    df_combined = df_combined[~df_combined.index.isin(first_occur_df.sort_values(['horse_id', 'run_date']).set_index(['horse_id', 'run_date']).index)]
except FileNotFoundError:
    horse_race_df = pd.read_csv(file_directory + 'data/horse_race.csv', low_memory=False, index_col=0)
    horse_race_df['age_int'] = horse_race_df['sex_age'].apply(lambda x: re.search(r'\d+', x).group(0)).astype(int)
    horse_race_df['run_date'] = horse_race_df['run_date'].apply(pd.Timestamp)
    horse_race_df = horse_race_df.sort_values(['horse_id', 'run_date'])
    drop_cols(horse_race_df)
    
    df_combined = horse_race_df.set_index(['horse_id', 'run_date'])
    df_combined['run_time_diff'] = df_combined['run_time_1000'].diff()
    df_combined['last_run_time'] = df_combined['run_time_1000'] - df_combined['run_time_diff']
    df_combined['run_time_quo'] = df_combined['run_time_1000'] / df_combined['last_run_time']
    df_combined['run_time_mean'] = df_combined['last_run_time']

    df_reset = df_combined['run_time_mean'].reset_index()
    horse_id_lst = list(df_reset['horse_id'])
    run_time_mean_lst = list(df_reset['run_time_mean'])
    new_run_time_mean_lst = []
    new_run_time_median_lst = []
    curr_index = horse_id_lst[0]
    curr_count = 0
    curr_sum = 0
    curr_stored = []
    for index, value in zip(horse_id_lst, run_time_mean_lst):
        if index != curr_index:
            curr_count = 1
            curr_sum = value
            curr_index = index
            curr_stored = [value]
        else:
            curr_count += 1
            curr_sum += value
            curr_stored.append(value)
        new_run_time_mean_lst.append(curr_sum / (curr_count * 1.0))
        new_run_time_median_lst.append(np.median(curr_stored))
    df_combined['run_time_mean'] = pd.Series(new_run_time_mean_lst, index=df_combined.index)
    df_combined['run_time_median'] = pd.Series(new_run_time_median_lst, index=df_combined.index)

    for window in range(2, max_window + 1):
        ma = df_combined.groupby(level=0)['run_time_1000'].rolling(window).mean().groupby(level=0).shift(1)
        ma = ma.reset_index(level=1)['run_time_1000'].reset_index()
        ewma = df_combined.groupby(level=0)['run_time_1000'].apply(lambda series: series.ewm(ignore_na=True, 
                                                                                             min_periods=window, 
                                                                                             adjust=True,
                                                                                             com=0.030927835051546).mean())
        ewma = ewma.groupby(level=0).shift(1)
        df_combined['run_time_ma_window_%s' % str(window)] = ma.set_index(['horse_id', 'run_date'])['run_time_1000']
        df_combined['run_time_ewma_window_%s' % str(window)] = ewma
    df_combined.reset_index().to_csv('data/horse_race_combined.csv', encoding='utf-8')

In [ ]:
dependent = ['run_time_1000',
             'run_time_diff', 'run_time_quo', 
             'run_time_mean', 'run_time_median'] + \
            ['run_time_ma_window_%s' % str(idx) for idx in range(2, max_window + 1)] + \
            ['run_time_ewma_window_%s' % str(idx) for idx in range(2, max_window + 1)]
df_combined_y = df_combined[dependent].copy()
df_combined_x = df_combined[list(filter(lambda x: x not in dependent, df_combined.columns))].copy()
df_y_original_dict = {}
df_y_original_dict['run_time_diff'] = df_combined_x['last_run_time']
df_y_original_dict['run_time_quo'] = df_combined_x['last_run_time']
for col_name in dependent[3:]:
    df_combined_y[col_name + '_diff'] = df_combined_y['run_time_1000'] - df_combined_y[col_name]
    df_combined_y[col_name + '_quo'] = df_combined_y['run_time_1000'] / df_combined_y[col_name]
    df_y_original_dict[col_name + '_diff'] = df_combined_y[col_name]
    df_y_original_dict[col_name + '_quo'] = df_combined_y[col_name]
    df_combined_y.drop(col_name, axis=1, inplace=True)
df_combined_y[list(filter(lambda x: 'diff' in x or 'quo' in x, df_combined_y.columns))].T

## 5.1 OLS Trial Run

In [ ]:
# Model testing for run time residual
x = feature_engineer(df_combined_x.reset_index(), 'df_combined_all' if not sample else 'df_sampled')
y = df_combined_y.loc[df_combined_y.index.isin(x.index), 'run_time_diff']

# OLS
reg = LinearRegression(fit_intercept=True)
scores_reg = cross_val_score(reg, x, y, scoring='neg_mean_squared_error')
print("RMSE for OLS: %0.5f" % np.sqrt(-scores_reg.mean()))

## 5.2 Baseline Regression Models and Stacking

In [ ]:
try:
    report_directory = file_directory + 'predictive/report/'
    
    # Initiate class object
    mc = ModelComparer(df_combined_x, df_combined_y, df_y_original_dict, sampled=sample, random_state=37, ratio=0.8)
    r_square_score = make_scorer(ModelComparer.get_r_squared, greater_is_better=True)
    rmse_score = make_scorer(ModelComparer.get_rmse, greater_is_better=False)
    
    rmse_report = pd.read_csv(report_directory + 'rmse_report_baseline.csv', index_col=0)
    rsquare_report = pd.read_csv(report_directory + 'rsquare_report_baseline.csv', index_col=0)
    meta_report = pd.read_csv(report_directory + 'meta_report_baseline.csv', index_col=0)
except FileNotFoundError:
    # Add base model
    mc.add_model(LinearRegression, model_name='OLS - Base Model', fit_intercept=True)
    mc.add_model(XGBRegressor, 'XGB - Base Model (0.1LR)', learning_rate=0.1)
    mc.add_model(DecisionTreeRegressor, 'DT - Base Model (6MD)', max_depth=6)
    mc.add_model(RandomForestRegressor, 'RF - Base Model (6MD, 20E)', max_depth=6, n_estimators=20)
    mc.add_model(GradientBoostingRegressor, 'GBM - Base Model (6MD, 0.1LR)', 
                 n_estimators=100, max_depth=6, min_samples_split=2, learning_rate=0.1)

    # Add ANN with verbose as True
    mc.add_model(MLPRegressor, 'ANN - Base Model (normalized, 100000MI, 0.01LR, 300HL)',
                 max_iter=100000, learning_rate_init=0.01, hidden_layer_sizes=(300,), activation='tanh', 
                 verbose=True)

    # Stacking model
    ols_base = LinearRegression(fit_intercept=True)
    xgb_base = XGBRegressor(learning_rate=0.1)
    dt_base = DecisionTreeRegressor(max_depth=6)
    rf_base = RandomForestRegressor(max_depth=6, n_estimators=20)
    gbm_base = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_split=2, learning_rate=0.1)
    ann_base = MLPRegressor(max_iter=100000, learning_rate_init=0.01, hidden_layer_sizes=(300,), activation='tanh',  
                            verbose=False)
    xgb_meta = XGBRegressor(learning_rate=0.1)
    mc.add_model(StackingRegressor, 'Meta - XGB (0.1LR)',
                 regressors=[ols_base, xgb_base, dt_base, rf_base, gbm_base, ann_base], 
                 meta_regressor=xgb_meta)

    # Get model report
    rmse_report = mc.get_report(filter_word='RMSE')
    rsquare_report = mc.get_report(filter_word='R^2')
    rmse_report.to_csv(file_directory + 'predictive/report/rmse_report_baseline.csv')
    rsquare_report.to_csv(file_directory + 'predictive/report/rsquare_report_baseline.csv')
    
    meta_report = mc.get_meta_report()
    meta_report.to_csv(file_directory + 'predictive/report/meta_report_baseline.csv')

In [ ]:
# Set some specs for plotting
mpl.rcParams['figure.figsize'] = (16.0, 8.0)
mpl.style.use('ggplot')
sns.heatmap(meta_report.reset_index().set_index(['model_name', 'index']).astype(float), annot=True, fmt='.2f', cmap='RdYlBu_r')

In [ ]:
value_df = rmse_report.dropna().astype(float).iloc[:, :]
value_df = value_df.applymap(lambda x: float(x))
sns.heatmap(value_df, annot=True, fmt='g', cmap='RdYlBu_r') 

In [ ]:
value_df = rmse_report.dropna().astype(float).iloc[:, 1:]
value_df = value_df.applymap(lambda x: float(x))
sns.heatmap(value_df, annot=True, fmt='g', cmap='RdYlBu_r') 

In [ ]:
value_df = rsquare_report.dropna().astype(float).iloc[:, :]
value_df = value_df.applymap(lambda x: float(x))
sns.heatmap(value_df, annot=True, fmt='g', cmap='RdYlBu')

## 5.3 Grid Search

### 5.3.1 Artificial Nerural Network Hyper-parameter Tuning 

In [ ]:
ann_tuned_params = {
    'activation': ['relu', 'tanh'], 
    'learning_rate_init': [0.01, 0.001, 0.002], 
    'max_iter': [100000, ], 
    'hidden_layer_sizes': [(100, ), (150, ), (200, ), (250, ), (300, ), (350, )]
}
get_best_model(mc, 'ann', MLPRegressor, ann_tuned_params, n_jobs=1, randomized=False,
               scoring=rmse_score, verbose=2,
               filter_func=lambda x: 'run_time_1000' in x)

### 5.3.2 Random Forest Hyper-parameter Tuning

In [ ]:
rf_tuned_params = {
    'n_estimators': np.arange(20, 70, 10), 
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': np.arange(5, 10, 1)
}

get_best_model(mc, 'rf', RandomForestRegressor, rf_tuned_params, n_jobs=-1, randomized=True,
               scoring=rmse_score, verbose=2, 
               filter_func=lambda x: 'run_time_1000' in x)

### 5.3.3 XGBoost Hyper-parameter Tuning

In [ ]:
xgb_tuned_params = {
    'max_depth': np.arange(3, 11, 2),
    'subsample': np.arange(0.6, 1.1, 0.1),
    'colsample_bytree': np.arange(0.6, 1.2, 0.2),
    'min_child_weight': np.arange(1, 11, 3),
    'gamma': np.arange(0, 0.5, 1),
    'reg_alpha': [0, 0.001, 0.01]
} 
get_best_model(mc, 'xgb', XGBRegressor, xgb_tuned_params, n_jobs=-1, randomized=True,
               scoring=rmse_score, verbose=2, 
               filter_func=lambda x: 'run_time_1000' in x)

### 5.3.4 Gradient Boost Machine Hyper-parameter Tuning

In [ ]:
gbm_tuned_params = {
    'n_estimators': np.arange(100, 220, 20),
    'max_depth': np.arange(5, 12, 2),
    'min_samples_split': np.arange(100, 1000, 200),
    'min_samples_leaf': np.arange(30, 90, 20),
    'max_features': np.arange(20, 50, 10),
    'subsample': np.arange(0.6, 1, 0.1)
}
get_best_model(mc, 'gbm', GradientBoostingRegressor, gbm_tuned_params, n_jobs=-1, randomized=True,
               scoring=rmse_score, verbose=2, 
               filter_func=lambda x: 'run_time_1000' in x)

## 5.4 OLS Stacking Model with Grid Search Cross-Validation

In [ ]:
# Stacking model trial code (disable the comment to start)
if 1 == 0:
    ols = LinearRegression(fit_intercept=False)
    xgb = XGBRegressor(learning_rate=0.1)
    dt = DecisionTreeRegressor()
    rf = RandomForestRegressor()
    ann = MLPRegressor(max_iter=100000, learning_rate_init=0.01, activation='tanh')
    gbm = GradientBoostingRegressor(learning_rate=0.1)

    stack_tuned_params = {'xgbregressor__max_depth': [6, 9],
                          'xgbregressor__min_child_weight': [3, 10],
                          'xgbregressor__subsample': [0.8, 1],        
                          'randomforestregressor__n_estimators': [20, 40], 
                          'randomforestregressor__max_depth': [5, 6],
                          'mlpregressor__hidden_layer_sizes': [(100, ), (200, ), (250, ), (350, )],
                          'gradientboostingregressor__n_estimators': [20, 50, 100],
                          'gradientboostingregressor__max_features': [20, 30, 'auto'],
                          'gradientboostingregressor__subsample': [0.8, 1]}
    ols_meta = LinearRegression(fit_intercept=True)
    get_best_model(mc, 'ols_stack', 
                   model_method=StackingCVRegressor, 
                   tuned_params=stack_tuned_params, 
                   scoring=rmse_score, verbose=1,
                   regressors=[ols, xgb, dt, rf, ann, gbm], 
                   meta_regressor=ols_meta, 
                   use_features_in_secondary=True)